# LLM Optimization Modelling Experiment

In [3]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Image
from IPython.display import Markdown

## 1. Define the problem description

In [88]:
problem = '''You are a city planner, looking to open facilities at some locations. We have a set of customers and a set of possible locations for opening facilities. Each potential location for establishing a facility incurs a fixed annual activation cost, which must be paid if the facility is used, regardless of the service volume it handles. Furthermore, this service volume at each facility is capped at a maximum annual limit. Additionally, there are transportation costs associated with servicing each customer from each facility.
The goal is to minimize the overall costs, which include both the fixed activation costs for any opened facilities and the transportation costs for servicing customers. This must be done while making sure that each customer’s demand is met, each facility does not exceed its maximum service volume, and customers can of course only be serviced by a facility that is opened. 
Please formulate this as a mathematical optimization model.'''

## 2. Generate the mathematical model

In [89]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Please formulate a mathematical optimization model for this problem. Include parameters, decision variables, the objective function and the constraints in your answer.
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [90]:
#Show the resopnse in a formatted way
Markdown(response.text)

## Mathematical Optimization Model for Facility Location and Customer Allocation

**Sets:**

*  $I$: Set of potential facility locations.
*  $J$: Set of customers. 

**Parameters:**

*  $f_i$: Fixed annual activation cost for facility $i \in I$.
*  $C_i$: Maximum annual service volume of facility $i \in I$.
*  $d_j$: Annual demand of customer $j \in J$.
*  $t_{ij}$: Transportation cost per unit demand of serving customer $j \in J$ from facility $i \in I$.

**Decision Variables:**

*  $x_{ij}$: Amount of demand of customer $j \in J$ served by facility $i \in I$.
*  $y_i$: Binary variable equal to 1 if facility $i \in I$ is opened, 0 otherwise.

**Objective Function:**

Minimize the total cost, which is the sum of fixed activation costs and transportation costs:

$$
\min \sum_{i \in I} f_i y_i + \sum_{i \in I} \sum_{j \in J} t_{ij} x_{ij}
$$

**Constraints:**

1. **Demand satisfaction:** Each customer's demand must be met:
   $$
   \sum_{i \in I} x_{ij} = d_j, \quad \forall j \in J
   $$

2. **Capacity constraint:** No facility can exceed its maximum service volume:
   $$
   \sum_{j \in J} x_{ij} \leq C_i y_i, \quad \forall i \in I
   $$

3. **Service only from opened facilities:** Customers can only be served by facilities that are open:
   $$
   x_{ij} \leq d_j y_i, \quad \forall i \in I, \ \forall j \in J 
   $$

4. **Binary constraint for facility activation:**
   $$
   y_i \in \{0, 1\}, \quad \forall i \in I
   $$

5. **Non-negativity constraint for service volume:**
   $$
   x_{ij} \geq 0, \quad \forall i \in I, \ \forall j \in J
   $$

This model is a Mixed-Integer Linear Program (MILP) because it involves both continuous variables ($x_{ij}$) and binary variables ($y_i$). It can be solved using optimization software packages to determine the optimal facility locations and customer allocation that minimizes overall costs. 


## 3. Generate the pyomo code

In [91]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt2 += response.text 
response2 = generative_multimodal_model.generate_content([prompt2])

In [92]:
#Showing the code in a formatted way
Markdown(response2.text)

```python
import pyomo.environ as pyo

# Sample Data (replace with actual data)
I = ['Facility1', 'Facility2', 'Facility3']  
J = ['Customer1', 'Customer2', 'Customer3', 'Customer4'] 

f = {'Facility1': 100, 'Facility2': 150, 'Facility3': 120} # Sample data
C = {'Facility1': 500, 'Facility2': 600, 'Facility3': 550} # Sample data 
d = {'Customer1': 100, 'Customer2': 150, 'Customer3': 200, 'Customer4': 180} # Sample data
t = {('Facility1', 'Customer1'): 2, ('Facility1', 'Customer2'): 3, ('Facility1', 'Customer3'): 5, ('Facility1', 'Customer4'): 4, 
     ('Facility2', 'Customer1'): 4, ('Facility2', 'Customer2'): 2, ('Facility2', 'Customer3'): 3, ('Facility2', 'Customer4'): 5,
     ('Facility3', 'Customer1'): 3, ('Facility3', 'Customer2'): 4, ('Facility3', 'Customer3'): 2, ('Facility3', 'Customer4'): 3} # Sample data 

# Model
model = pyo.ConcreteModel()

# Sets
model.I = pyo.Set(initialize=I)
model.J = pyo.Set(initialize=J)

# Parameters
model.f = pyo.Param(model.I, initialize=f)
model.C = pyo.Param(model.I, initialize=C)
model.d = pyo.Param(model.J, initialize=d)
model.t = pyo.Param(model.I, model.J, initialize=t)

# Decision Variables
model.x = pyo.Var(model.I, model.J, within=pyo.NonNegativeReals)
model.y = pyo.Var(model.I, within=pyo.Binary)

# Objective Function
def objective_rule(model):
    return sum(model.f[i] * model.y[i] for i in model.I) + sum(model.t[i, j] * model.x[i, j] for i in model.I for j in model.J)
model.obj = pyo.Objective(rule=objective_rule, sense=pyo.minimize)

# Constraints
def demand_rule(model, j):
    return sum(model.x[i, j] for i in model.I) == model.d[j]
model.demand_constraint = pyo.Constraint(model.J, rule=demand_rule)

def capacity_rule(model, i):
    return sum(model.x[i, j] for j in model.J) <= model.C[i] * model.y[i]
model.capacity_constraint = pyo.Constraint(model.I, rule=capacity_rule)

def service_rule(model, i, j):
    return model.x[i, j] <= model.d[j] * model.y[i]
model.service_constraint = pyo.Constraint(model.I, model.J, rule=service_rule)

# Solve the model
solver = pyo.SolverFactory('glpk') 
solver.solve(model)

# Print the results
print("Optimal Solution:")
print("Objective value:", pyo.value(model.obj))
for i in model.I:
    if pyo.value(model.y[i]) > 0.5:
        print(f"Facility {i} is opened.")
        for j in model.J:
            if pyo.value(model.x[i, j]) > 0:
                print(f"  Serving {pyo.value(model.x[i, j])} units of demand for Customer {j}")
```

This code defines the model as a ConcreteModel in Pyomo, sets up the sets, parameters, decision variables, objective function, and constraints as specified in the mathematical formulation. Then, it uses the 'glpk' solver to find the optimal solution and prints the results, including the objective value, which facilities are open, and the amount of demand served for each customer from each facility. 

Remember to replace the sample data with your actual data.


## 4. Input problem data and try running the generated code

In [94]:
import pyomo.environ as pyo

# Sample Data (replace with actual data)
I = ['Facility1', 'Facility2', 'Facility3']  
J = ['Customer1', 'Customer2', 'Customer3', 'Customer4', 'Customer5']

f = {'Facility1': 1000, 'Facility2': 1000, 'Facility3': 1000}   # Sample data
C = {'Facility1': 500, 'Facility2': 500, 'Facility3': 500} # Sample data 
d = {'Customer1': 80, 'Customer2': 270, 'Customer3': 250, 'Customer4': 160, 'Customer5': 180} # Sample data
t = {
    ('Facility1', 'Customer1'): 4,
    ('Facility1', 'Customer2'): 5,
    ('Facility1', 'Customer3'): 6,
    ('Facility1', 'Customer4'): 8,
    ('Facility1', 'Customer5'): 10,
    ('Facility2', 'Customer1'): 6,
    ('Facility2', 'Customer2'): 4,
    ('Facility2', 'Customer3'): 3,
    ('Facility2', 'Customer4'): 5,
    ('Facility2', 'Customer5'): 8,
    ('Facility3', 'Customer1'): 9,
    ('Facility3', 'Customer2'): 7,
    ('Facility3', 'Customer3'): 4,
    ('Facility3', 'Customer4'): 3,
    ('Facility3', 'Customer5'): 4,
}
# Model
model = pyo.ConcreteModel()

# Sets
model.I = pyo.Set(initialize=I)
model.J = pyo.Set(initialize=J)

# Parameters
model.f = pyo.Param(model.I, initialize=f)
model.C = pyo.Param(model.I, initialize=C)
model.d = pyo.Param(model.J, initialize=d)
model.t = pyo.Param(model.I, model.J, initialize=t)

# Decision Variables
model.x = pyo.Var(model.I, model.J, within=pyo.NonNegativeReals)
model.y = pyo.Var(model.I, within=pyo.Binary)

# Objective Function
def objective_rule(model):
    return sum(model.f[i] * model.y[i] for i in model.I) + sum(model.t[i, j] * model.x[i, j] for i in model.I for j in model.J)
model.obj = pyo.Objective(rule=objective_rule, sense=pyo.minimize)

# Constraints
def demand_rule(model, j):
    return sum(model.x[i, j] for i in model.I) == model.d[j]
model.demand_constraint = pyo.Constraint(model.J, rule=demand_rule)

def capacity_rule(model, i):
    return sum(model.x[i, j] for j in model.J) <= model.C[i] * model.y[i]
model.capacity_constraint = pyo.Constraint(model.I, rule=capacity_rule)

def service_rule(model, i, j):
    return model.x[i, j] <= model.d[j] * model.y[i]
model.service_constraint = pyo.Constraint(model.I, model.J, rule=service_rule)

# Solve the model
solver = pyo.SolverFactory('glpk') 
solver.solve(model)

# Print the results
print("Optimal Solution:")
print("Objective value:", pyo.value(model.obj))
for i in model.I:
    if pyo.value(model.y[i]) > 0.5:
        print(f"Facility {i} is opened.")
        for j in model.J:
            if pyo.value(model.x[i, j]) > 0:
                print(f"  Serving {pyo.value(model.x[i, j])} units of demand for Customer {j}")

Optimal Solution:
Objective value: 5610.0
Facility Facility2 is opened.
  Serving 80.0 units of demand for Customer Customer1
  Serving 270.0 units of demand for Customer Customer2
  Serving 150.0 units of demand for Customer Customer3
Facility Facility3 is opened.
  Serving 100.0 units of demand for Customer Customer3
  Serving 160.0 units of demand for Customer Customer4
  Serving 180.0 units of demand for Customer Customer5


## 5. Correct the code to verify model viability (optional)

## 6. Printing the outputs as strings, so they can be saved.

In [95]:
print(response.text)

## Mathematical Optimization Model for Facility Location and Customer Allocation

**Sets:**

*  $I$: Set of potential facility locations.
*  $J$: Set of customers. 

**Parameters:**

*  $f_i$: Fixed annual activation cost for facility $i \in I$.
*  $C_i$: Maximum annual service volume of facility $i \in I$.
*  $d_j$: Annual demand of customer $j \in J$.
*  $t_{ij}$: Transportation cost per unit demand of serving customer $j \in J$ from facility $i \in I$.

**Decision Variables:**

*  $x_{ij}$: Amount of demand of customer $j \in J$ served by facility $i \in I$.
*  $y_i$: Binary variable equal to 1 if facility $i \in I$ is opened, 0 otherwise.

**Objective Function:**

Minimize the total cost, which is the sum of fixed activation costs and transportation costs:

$$
\min \sum_{i \in I} f_i y_i + \sum_{i \in I} \sum_{j \in J} t_{ij} x_{ij}
$$

**Constraints:**

1. **Demand satisfaction:** Each customer's demand must be met:
   $$
   \sum_{i \in I} x_{ij} = d_j, \quad \forall j \in J
   $$

In [96]:
print(response2.text)

```python
import pyomo.environ as pyo

# Sample Data (replace with actual data)
I = ['Facility1', 'Facility2', 'Facility3']  
J = ['Customer1', 'Customer2', 'Customer3', 'Customer4'] 

f = {'Facility1': 100, 'Facility2': 150, 'Facility3': 120} # Sample data
C = {'Facility1': 500, 'Facility2': 600, 'Facility3': 550} # Sample data 
d = {'Customer1': 100, 'Customer2': 150, 'Customer3': 200, 'Customer4': 180} # Sample data
t = {('Facility1', 'Customer1'): 2, ('Facility1', 'Customer2'): 3, ('Facility1', 'Customer3'): 5, ('Facility1', 'Customer4'): 4, 
     ('Facility2', 'Customer1'): 4, ('Facility2', 'Customer2'): 2, ('Facility2', 'Customer3'): 3, ('Facility2', 'Customer4'): 5,
     ('Facility3', 'Customer1'): 3, ('Facility3', 'Customer2'): 4, ('Facility3', 'Customer3'): 2, ('Facility3', 'Customer4'): 3} # Sample data 

# Model
model = pyo.ConcreteModel()

# Sets
model.I = pyo.Set(initialize=I)
model.J = pyo.Set(initialize=J)

# Parameters
model.f = pyo.Param(model.I, initialize=f)
model.C 